In [1]:
!pip install torchtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!python -m spacy download en_core_web_sm

2023-02-23 14:47:54.950253: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 14:47:55.814301: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-23 14:47:55.814412: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-23 14:47:55.814431: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [ ]:
# !pip install tokenizers

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext, datasets, math
from tqdm import tqdm

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#make our work comparable if restarted the kernel
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# torch.cuda.get_device_name(0)

cuda


1. Load data - https://huggingface.co/datasets/codeparrot/github-jupyter-code-to-text

In [7]:
# dataset =  datasets.load_dataset("codeparrot/github-jupyter-code-to-text")
training_data = datasets.load_dataset("codeparrot/github-jupyter-code-to-text", split="train")
testing_data = datasets.load_dataset("codeparrot/github-jupyter-code-to-text", split="test")

# print(train_data.shape)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/codeparrot___parquet/codeparrot--github-jupyter-code-to-text-cf9b56d996fd17e1/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


In [8]:
# brief look at the dataset
print(testing_data.shape)

(11864, 4)


In [9]:
print(training_data[2])

{'repo_name': 'ganguli-lab/twpca', 'path': 'notebooks/warp_unit_tests.ipynb', 'license': 'mit', 'content': '_, _, data = twpca.datasets.jittered_neuron()\nmodel = TWPCA(data, n_components=1, warpinit=\'identity\')\n\nnp.all(np.isclose(model.params[\'warp\'], np.arange(model.shared_length), atol=1e-5, rtol=2))\n\nnp.nanmax(np.abs(model.transform() - data)) < 1e-5\n\n"""\nExplanation: check identity warp does not change data appreciably\nEnd of explanation\n"""\n\n\nmodel = TWPCA(data, n_components=1, warpinit=\'shift\')\n\nplt.imshow(np.squeeze(model.transform()))\n\n"""\nExplanation: check that shift initialization for warp solves the simple toy problem\nEnd of explanation\n"""\n\n\n'}


In [10]:
print(training_data)

Dataset({
    features: ['repo_name', 'path', 'license', 'content'],
    num_rows: 47452
})


In [11]:
# let look at the content
print(training_data['content'][0])

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

"""
Explanation: Simple MNIST convnet
Author: fchollet<br>
Date created: 2015/06/19<br>
Last modified: 2020/04/21<br>
Description: A simple convnet that achieves ~99% test accuracy on MNIST.
Setup
End of explanation
"""


# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test

In [12]:
# split each line

training = [split for text in training_data['content'] for split in text.split('\n') if split != ""]
testing = [split for text in testing_data['content'] for split in text.split('\n') if split != ""]

In [13]:
print(training[2])

from tensorflow.keras import layers


In [14]:
# it's too much

print(len(training))
print(len(testing))

# need to cut the data size

11367363
2875424


In [15]:
from spacy.lang.en.stop_words import STOP_WORDS
import spacy
import re
import en_core_web_sm

nlp = en_core_web_sm.load()


# this function I borrow from TonsonP / Ati
def preprocessing(sentence):
    
    # Clear the html tag by using regular expression.
    sentence = re.sub("<[^>]*>", "", sentence) # RegEx to remove html tags
    sentence = re.sub("[^\x00-\x7F]+", "", sentence) # RegEx to remove another languages that is not English.
    stopwords = list(STOP_WORDS)
    doc = nlp(sentence)
    cleaned_tokens = []
    
    # Cleaned its with spacy, remove symbol, punct and empty space
    # It's smart enough to not remove the . in the middle of sentences. E.g. "np.array"

    for token in doc: 
        if token.text not in stopwords and token.pos_ != 'PUNCT' and token.pos_ != 'SPACE' and \
            token.pos_ != 'SYM' and token.pos_!= 'X':
                cleaned_tokens.append(token.lemma_.lower().strip())
                
    return " ".join(cleaned_tokens)

In [16]:
from torchtext.data.utils import get_tokenizer

def yield_tokens(data_iter):
    for text in data_iter:
        text = preprocessing(text)
        yield tokenizer(text)

tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

tokenized_dataset_train = yield_tokens(training[:100000])
tokenized_dataset_test = yield_tokens(testing[:25000])

In [17]:
## numericalizing
from torchtext.vocab import build_vocab_from_iterator

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(training[:100000]), min_freq=5) 
vocab.insert_token('<unk>', 0)           
vocab.insert_token('<eos>', 1)            
vocab.set_default_index(vocab['<unk>'])   
print(len(vocab))                         
print(vocab.get_itos()[:100])

9442
['<unk>', '<eos>', '"', ',', ')', '=', ':', 'the', '#', "'", '.', 'of', '    ', '(', '-', ']', ' ', 'to', 'a', 'in', 'and', 'explanation', 'End', 'Explanation', 'for', '        ', 'is', 'as', 'we', '/', '[', '$', '}', '):', 'import', 'that', 'data', 'from', 'with', '1', 'PROPERTY', '     ', '+', '*', '>', '!', 'The', 'this', 'are', 'be', 'model', 'can', 'you', ';', 'if', '0', '            ', 'it', '2', 'on', 'TODO', 'will', '  ', '<', 'Set', '_', '%', 'Is', '{', 'follows', 'def', 'ID', 'please', 'NOT', 'enter', 'DO', 'EDIT', 'value(s', 'return', 'by', 'Required', 'x', "'s", 'True', 'We', 'an', 'use', 'or', 'not', 'function', 'VALUE', 'DOC.set_value("value', 'each', 'using', 'have', '3', 'This', 'our', 'time', 'which']


In [18]:
# use another language basic_english, but in the next step, we have a problem when adding vocab.insert_token('<unk>', 0). I think that is because <unk> is not basic english
# tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

# #function to tokenize
# tokenize_data = lambda example, tokenizer: {'tokens': tokenizer(example['content'])}  

# #map the function to each example
# tokenized_train = train_data.map(tokenize_data, remove_columns=['content'], fn_kwargs={'tokenizer': tokenizer})

# tokenized_test = test_data.map(tokenize_data, remove_columns=['content'], fn_kwargs={'tokenizer': tokenizer})

In [19]:
def get_data(dataset, vocab, batch_size):
    data = []                                                   
    for example in dataset:       
        #appends eos so we know it ends....so model learn how to end...                             
        tokens = example.append('<eos>') #end of sentence
        #numericalize          
        tokens = [vocab[token] for token in example] 
        data.extend(tokens)                                    
    data = torch.LongTensor(data)                                 
    num_batches = data.shape[0] // batch_size 
    data = data[:num_batches * batch_size]                       
    data = data.view(batch_size, num_batches)        
    return data

In [20]:
batch_size = 128
train_data = get_data(tokenized_dataset_train, vocab, batch_size)
valid_data = get_data(tokenized_dataset_test, vocab, batch_size)

In [21]:
train_data.shape

torch.Size([128, 3789])

In [22]:
class LSTMLanguageModel(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, num_layers, dropout_rate):
                
        super().__init__()
        self.hid_dim = hid_dim
        self.num_layers = num_layers
        self.embedding = nn.Embedding(vocab_size,emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, num_layers=num_layers, 
                                        dropout = dropout_rate, batch_first = True)
        self.dropout = nn.Dropout(dropout_rate)
        #when you do LM, you look forward, so it does not make sense to do bidirectional
        self.fc = nn.Linear(hid_dim,vocab_size)

    def init_hidden(self, batch_size, device):
        #this function gonna be run in the beginning of the epoch
        hidden = torch.zeros(self.num_layers, batch_size, self.hid_dim).to(device)
        cell   = torch.zeros(self.num_layers, batch_size, self.hid_dim).to(device)

        return hidden, cell #return as tuple

    def detach_hidden(self, hidden):
        hidden, cell = hidden
        hidden = hidden.detach() #removing this hidden from gradients graph
        cell =  cell.detach() #removing this hidden from gradients graph
        return hidden, cell

    def forward(self, src, hidden):
        #src: [batch_size, seq_len]

        #embed 
        embedded = self.embedding(src)
        #embed : [batch_size, seq_len, emb_dim]

        #send this to the lstm
        #we want to put hidden here... because we want to reset hidden .....
        output, hidden = self.lstm(embedded, hidden)
        #output : [batch_size, seq_len, hid_dim] ==> all hidden states
        #hidden : [batch_size, seq_len, hid_dim] ==> last hidden states from each layer

        output = self.dropout(output)
        prediction = self.fc(output)
        #prediction: [batch size, seq_len, vocab_size]
        return prediction, hidden

In [23]:
vocab_size = len(vocab)
emb_dim = 1024                # 400 in the paper
hid_dim = 1024                # 1150 in the paper
num_layers = 2                # 3 in the paper
dropout_rate = 0.65              
lr = 1e-3

In [24]:
model = LSTMLanguageModel(vocab_size, emb_dim, hid_dim, num_layers, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

The model has 36,140,258 trainable parameters


In [25]:
def get_batch(data, seq_len, idx):
    #data #[batch size, bunch of tokens]
    src    = data[:, idx:idx+seq_len]                   
    target = data[:, idx+1:idx+seq_len+1]  #target simply is ahead of src by 1            
    return src, target

In [26]:
def train(model, data, optimizer, criterion, batch_size, seq_len, clip, device):
    
    epoch_loss = 0
    model.train()
    # drop all batches that are not a multiple of seq_len
    # data #[batch size, bunch of tokens]
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]  #we need to -1 because we start at 0
    num_batches = data.shape[-1]
    
    #reset the hidden every epoch
    hidden = model.init_hidden(batch_size, device)
    
    for idx in tqdm(range(0, num_batches - 1, seq_len), desc='Training: ',leave=False):
        optimizer.zero_grad()
        
        #hidden does not need to be in the computational graph for efficiency
        hidden = model.detach_hidden(hidden)

        src, target = get_batch(data, seq_len, idx) #src, target: [batch size, seq len]
        src, target = src.to(device), target.to(device)
        batch_size = src.shape[0]
        prediction, hidden = model(src, hidden)               

        #need to reshape because criterion expects pred to be 2d and target to be 1d
        prediction = prediction.reshape(batch_size * seq_len, -1)  #prediction: [batch size * seq len, vocab size]  
        target = target.reshape(-1)
        loss = criterion(prediction, target)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item() * seq_len
    return epoch_loss / num_batches

In [27]:
def evaluate(model, data, criterion, batch_size, seq_len, device):

    epoch_loss = 0
    model.eval()
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)

    with torch.no_grad():
        for idx in range(0, num_batches - 1, seq_len):
            hidden = model.detach_hidden(hidden)
            src, target = get_batch(data, seq_len, idx)
            src, target = src.to(device), target.to(device)
            batch_size= src.shape[0]

            prediction, hidden = model(src, hidden)
            prediction = prediction.reshape(batch_size * seq_len, -1)
            target = target.reshape(-1)

            loss = criterion(prediction, target)
            epoch_loss += loss.item() * seq_len
    return epoch_loss / num_batches

In [28]:
n_epochs = 25
seq_len  = 50 #<----decoding length
clip    = 0.25

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

best_valid_loss = float('inf')

for epoch in range(n_epochs):
    train_loss = train(model, train_data, optimizer, criterion, 
                batch_size, seq_len, clip, device)
    valid_loss = evaluate(model, valid_data, criterion, batch_size, 
                seq_len, device)

    lr_scheduler.step(valid_loss)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best-val-lstm_lm.pt')

    print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
    print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

	Train Perplexity: 167.214
	Valid Perplexity: 80.523


	Train Perplexity: 71.737
	Valid Perplexity: 65.144


	Train Perplexity: 56.875
	Valid Perplexity: 58.072


	Train Perplexity: 48.731
	Valid Perplexity: 53.665


	Train Perplexity: 42.642
	Valid Perplexity: 51.877


...

	Train Perplexity: 15.544
	Valid Perplexity: 45.766


	Train Perplexity: 15.546
	Valid Perplexity: 45.773


	Train Perplexity: 15.524
	Valid Perplexity: 45.773


	Train Perplexity: 15.529
	Valid Perplexity: 45.774


	Train Perplexity: 15.501
	Valid Perplexity: 45.775


In [29]:
def generate(prompt, max_seq_len, temperature, model, tokenizer, vocab, device, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    model.eval()
    tokens = tokenizer(prompt)
    indices = [vocab[t] for t in tokens]
    batch_size = 1
    hidden = model.init_hidden(batch_size, device)
    with torch.no_grad():
        for i in range(max_seq_len):
            src = torch.LongTensor([indices]).to(device)
            prediction, hidden = model(src, hidden)
            
            #prediction: [batch size, seq len, vocab size]
            #prediction[:, -1]: [batch size, vocab size] #probability of last vocab
            
            probs = torch.softmax(prediction[:, -1] / temperature, dim=-1)  
            prediction = torch.multinomial(probs, num_samples=1).item()    
            
            while prediction == vocab['<unk>']: #if it is unk, we sample again
                prediction = torch.multinomial(probs, num_samples=1).item()

            if prediction == vocab['<eos>']:    #if it is eos, we stop
                break

            indices.append(prediction) #autoregressive, thus output becomes input

    itos = vocab.get_itos()
    tokens = [itos[i] for i in indices]
    return tokens

In [36]:
prompt = 'import numpy as'
max_seq_len = 30
seed = 0
            #superdiverse   more diverse
temperatures = [0.5, 0.7, 0.75, 0.8, 1.0] 
#sample from this distribution higher probability will get more change
for temperature in temperatures:
    generation = generate(prompt, max_seq_len, temperature, model, tokenizer, 
                          vocab, device, seed)
    print(str(temperature)+'\n'+' '.join(generation)+'\n')

0.5
import numpy as

0.7
import numpy as tensorflow library tensorflow tensorflow create numpy package package create tensorflow graph

0.75
import numpy as result start library create numpy array contain 3 numpy array

0.8
import numpy as result start library not day what choice

1.0
import numpy as result start additional line file want analyze not day what choice

